In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
kgs.preallocate_matrices=False
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()
# model.models[1].model_FlatVel = seis_model.DummyModel()
# model.models[1].model_FlatVel.state=1
# model.models[1].model_Style_A = seis_model.DummyModel()
# model.models[1].model_Style_A.state=1
# model.models[1].model_TV2D = seis_model.DummyModel()
# model.models[1].model_TV2D.state=1
model.read_cache = False

In [6]:
%%time
data = kgs.load_all_train_data(validation_only=True)[::2]
model.models[1].run_in_parallel = True
model.models[1].model_Style_A.models = model.models[1].model_Style_A.models[0:1]
data_out = model.infer(data)

Inferring :  10%|██████▉                                                              | 25/250 [00:05<00:46,  4.82it/s]

capturing graph


Inferring :  10%|███████▏                                                             | 26/250 [00:06<00:57,  3.88it/s]

6.606731070343904e-05 6.606731070343904e-05


Inferring :  11%|███████▍                                                             | 27/250 [00:06<01:00,  3.68it/s]

...

5.222575799955865e-05 5.222575799955865e-05


Inferring :  61%|█████████████████████████████████████████▎                          | 152/250 [00:50<00:25,  3.80it/s]

3.455831806233989e-05 3.455831806233989e-05


Inferring : 100%|████████████████████████████████████████████████████████████████████| 250/250 [00:59<00:00,  4.24it/s]


CPU times: total: 56.1 s
Wall time: 1min


In [7]:
dfTimings = pd.read_csv(kgs.timing_filename)
dfTimings = dfTimings.groupby('family')['time_taken'].mean().to_frame('time_taken')

In [8]:
#kgs.disable_caching = True
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    try:
        res['time_taken'].append(dfTimings.loc[f].to_numpy()[0])
    except:
        res['time_taken'].append(0)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Importing cache brendan: 100%|███████████████████████████████████████████████████████| 25/25 [00:00<00:00, 4999.65it/s]



Brendan local score:  23.33086302485252
My local score:  7.995248444375797
Estimate public LB:  9.869465820991785


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.306740,3.306740,9.265849,0
1,CurveFault_B,74.582465,19.826906,39.138776,0
2,CurveVel_A,8.772963,4.332114,9.837551,0
3,CurveVel_B,26.563902,5.481069,16.276327,0
4,FlatFault_A,2.258997,2.258997,6.267089,0
5,FlatFault_B,27.190645,7.736392,17.294898,0
6,FlatVel_A,1.168041,0.041143,0.214286,0
7,FlatVel_B,4.845314,0.009714,0.085714,0
8,Style_A,37.007648,5.167781,9.543121,0
9,Style_B,47.611914,31.791628,40.132522,0


In [ ]:
# No extra model: 28.8
# FlatVel:
# StyleA: 

In [ ]:
28-30*0.081

In [ ]:
65000*0.09*0.15/4

In [ ]:
8